In [5]:
import pygmt
import numpy as np
import pandas as pd
import geopy.distance

In [61]:
colombia_stations = 'Colombia-Noise.csv'
ecuador_stations = 'Ecuador-Noise.csv'
earthquakes_file = 'Earthquakes.csv'
cities_file = 'Cities.csv'

In [91]:
# Read in station locations and noise for each station
dfE = pd.read_csv(ecuador_stations)
dfC = pd.read_csv(colombia_stations)
frames = [dfC, dfE ]
df = pd.concat(frames)

In [92]:
cities = pd.read_csv(cities_file)

In [93]:
eqs = pd.read_csv(earthquakes_file)
eqs['Date'] = pd.to_datetime(eqs['Date'])

In [94]:
#minimum number of stations for detection
min = 4
#Seimic velocity in km/sec
vel = 4
# Time required to process and issue warning
issue = 6

In [95]:
cities

,Latitude,Longitude,City
0,4.7110,-74.0721,Bogota
1,6.2476,-75.5658,Medellin
2,3.4516,-76.5320,Cali
3,11.0041,-74.8070,Barranquilla
4,10.3932,-75.4832,Cartagena


In [96]:
for i in cities.itertuples():
    eqs[i[3]+'_(dist) km'] = eqs.apply(lambda row: geopy.distance.geodesic( [i[1], i[2]],(row.Latitude,row.Longitude)).km, axis=1)
eqs       

,Latitude,Longitude,Depth,Date,Mw,Bogota_(dist) km,Medellin_(dist) km,Cali_(dist) km,Barranquilla_(dist) km,Cartagena_(dist) km
0,4.800,-77.180,19.1,1991-11-19,7.2,344.926904,240.034517,165.556988,734.304366,646.272064
1,4.720,-77.570,16.0,2004-11-15,7.2,388.076099,279.032038,181.539524,758.793228,668.320425
2,2.849,-74.798,55.0,2067-02-09,7.0,221.122487,385.357127,203.930822,901.891370,837.746081
3,0.955,-79.369,25.0,2006-01-31,8.8,720.565270,721.756729,419.270076,1220.443297,1128.838142
4,0.025,-79.955,25.0,2042-05-14,7.8,834.503536,843.357703,537.200829,1341.101194,1248.895466
5,1.000,-79.400,60.0,2058-01-19,7.8,720.531318,719.754710,418.629159,1217.333828,1125.550565
6,0.380,-79.950,20.0,2016-04-16,7.7,810.217131,811.279789,509.838994,1305.377431,1212.692305
7,2.320,-78.810,19.7,1979-12-12,8.1,589.064050,564.164332,282.487221,1057.292145,965.589259


In [97]:
def blind_spot(
        eq_lat,
        eq_lon,
        city_lat,
        city_lon,
        df,
        mini):
    df['distance']   = df.apply(lambda row: geopy.distance.geodesic((eq_lat,eq_lon),(row.lat,row.lon)).km, axis=1)
    df.sort_values(by='distance', inplace=True, ignore_index=True)
    dect_dist = df.loc[mini-1,'distance']
    return dect_dist

In [98]:
def warning_time(
        eq_lat,
        eq_lon,
        city_lat,
        city_lon,
        df,
        mini,
        velocity):
    df['distance']   = df.apply(lambda row: geopy.distance.geodesic((eq_lat,eq_lon),(row.lat,row.lon)).km, axis=1)
    df.sort_values(by='distance', inplace=True, ignore_index=True)
    dect_dist = df.loc[mini-1,'distance']
    city_dist = geopy.distance.geodesic((eq_lat,eq_lon),(city_lat,city_lon)).km
    warning_time_sec = (city_dist - dect_dist)/velocity
    return warning_time_sec

In [99]:
for i in cities.itertuples():
    eqs[i[3]+'_BlindSpot_(NoSim) km'] = eqs.apply(lambda row: blind_spot(row.Latitude, row.Longitude, i[1], i[2], df, min), axis=1)

In [100]:
for i in cities.itertuples():
    eqs[i[3]+'_Warning_(NoSim) sec'] = eqs.apply(lambda row: warning_time(row.Latitude, row.Longitude, i[1], i[2], df, min, vel), axis=1)

In [101]:
df25 = pd.read_csv('Sim-25km-Noise.csv')
frames = [dfC, dfE, df25]
df = pd.concat(frames)

In [102]:
for i in cities.itertuples():
    eqs[i[3]+'_BlindSpot_(25km) km'] = eqs.apply(lambda row: blind_spot(row.Latitude, row.Longitude, i[1], i[2], df, min), axis=1)

In [103]:
for i in cities.itertuples():
    eqs[i[3]+'_Warning_(25km) sec'] = eqs.apply(lambda row: warning_time(row.Latitude, row.Longitude, i[1], i[2], df, min, vel), axis=1)

In [104]:
df50 = pd.read_csv('Sim-50km-Noise.csv')
frames = [dfC, dfE, df50]
df = pd.concat(frames)

In [105]:
for i in cities.itertuples():
    eqs[i[3]+'_BlindSpot_(50km) km'] = eqs.apply(lambda row: blind_spot(row.Latitude, row.Longitude, i[1], i[2], df, min), axis=1)

In [106]:
for i in cities.itertuples():
    eqs[i[3]+'_Warning_(25km) sec'] = eqs.apply(lambda row: warning_time(row.Latitude, row.Longitude, i[1], i[2], df, min, vel), axis=1)

In [107]:
df100 = pd.read_csv('Sim-100km-Noise.csv')
frames = [dfC, dfE, df100]
df = pd.concat(frames)

In [108]:
for i in cities.itertuples():
    eqs[i[3]+'_BlindSpot_(100km) km'] = eqs.apply(lambda row: blind_spot(row.Latitude, row.Longitude, i[1], i[2], df, min), axis=1)

In [109]:
for i in cities.itertuples():
    eqs[i[3]+'_Warning_(100km) sec'] = eqs.apply(lambda row: warning_time(row.Latitude, row.Longitude, i[1], i[2], df, min, vel), axis=1)

In [110]:
eqs.to_csv('DetectionTable.csv', float_format="%.1f")